# Recommend System

## 개요

### 학습 목표

* 추천 시스템의 기본 개념에 대해 학습합니다.
* 추천 시스템의 종류에 대해 학습하고 사이킷런을 활용하여 연습해 봅니다.
* 실제 추천 시스템에는 더 많은 기술이 적용됨을 이해합니다.

### 목차

1. 들어가며
2. 추천 시스템 이란?
3. 유사도 계산
4. 추천 시스템 종류
* 콘텐츠 기반 필터링
* 협업 필터링
  * 사용자 기반
  * 아이템 기반
  * 잠재요인 협업 필터링
* 실제 추천 시스템

## 추천 시스템

* 범주형 데이터를 다룬다.
  - 영화 item 데이터와 user 데이터를 취급, 연속적이지 않고 이산적인 데이터. 이를 범주형 데이터
* (숫자 벡터로 변환한 뒤) 유사도를 계산한다.
  - 좌표로 나타내기 위해 숫자로 이루어진 벡터로 변환해야 한다. 그 거리를 계산하여 유사도를 계산.

## 코사인 유사도

### 설명

* 두 벡터의 방향이 이루는 각에 코사인을 취해 구함.
* 벡터의 방향이 동일한 경우 1
* 90도를 이루면 0
* 반대방향이면 -1
* -1 ~ 1 : 1에 가까울 수록 유사도가 높다.
* 두 벡터 A, B의 코사인 유사도 수식
* | 개수가 norm 숫자 = L2 Norm

$$ cosine similarity = cos(\theta) = \frac{A \cdot B}{||A||||B||} = \frac{\sum_{i=1}^{n}A_{i}B_{i}}{\sqrt{\sum_{i=1}^{n}A_{i}^{2}}\sqrt{\sum_{i=1}^{n}B_{i}^{2}}}$$

### Numpy 활용

In [1]:
import numpy as np

t1 = np.array([1, 1, 1])
t2 = np.array([2, 0, 1])

* [norm](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html) : 벡터의 경우 Frobenius norm을 구한다.

In [2]:
from numpy import dot
from numpy.linalg import norm
def cos_sim(A, B):
	return dot(A, B)/(norm(A)*norm(B))

In [3]:
cos_sim(t1, t2)

0.7745966692414834

### 사이킷런 활용

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
t1 = np.array([[1, 1, 1]])
t2 = np.array([[2, 0, 1]])
cosine_similarity(t1,t2)

array([[0.77459667]])

## 추천시스템의 종류

* 콘텐츠 기반 필터링(Content Based Filtering)
* 협업 필터링(Collaborative Filtering)
  - 사용자 기반
  - 아이템 기반
  - 잠재요인 협업 필터링 (latent factor collaborative filtering) → 행렬 인수분해(matrix factorization)
*  Deep Learning 적용 or Hybrid 방식

## 콘텐츠 기반 필터링

[참고](https://www.codeheroku.com/post.html?name=Building%20a%20Movie%20Recommendation%20Engine%20in%20Python%20using%20Scikit-Learn)

### 필요한 모듈 Import

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### 데이터 불러오기

In [7]:
import os
csv_path = 'data/movie_dataset.csv'
df = pd.read_csv(csv_path)
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


### 특성 선택

In [8]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

* 위 특성들 중에서 이것들만 사용하겠다.

In [9]:
features = ['keywords','cast','genres','director']
features

['keywords', 'cast', 'genres', 'director']

In [10]:
def combine_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']

combine_features(df[:5])

0    culture clash future space war space colony so...
1    ocean drug abuse exotic island east india trad...
2    spy based on novel secret agent sequel mi6 Dan...
3    dc comics crime fighter terrorist secret ident...
4    based on novel mars medallion space travel pri...
dtype: object

* 해당 특성을 텍스트 데이터로 보고, 보고자 하는 특성으로 이루어진 텍스트 데이터를 새로운 col로 생성

In [11]:
for feature in features:
    df[features] = df[features].fillna('')

df["combined_features"] = df.apply(combine_features,axis=1)
df["combined_features"]

0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6 Dan...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
                              ...                        
4798    united states\u2013mexico barrier legs arms pa...
4799     Edward Burns Kerry Bish\u00e9 Marsha Dietlein...
4800    date love at first sight narration investigati...
4801     Daniel Henney Eliza Coupe Bill Paxton Alan Ru...
4802    obsession camcorder crush dream girl Drew Barr...
Name: combined_features, Length: 4803, dtype: object

### 벡터화하고, 코사인 유사도를 계산합니다.

* CountVectorizer() : 등장 횟수를 벡터화

In [12]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])
print(type(count_matrix))
print(count_matrix.shape)
print(count_matrix)

<class 'scipy.sparse._csr.csr_matrix'>
(4803, 14845)
  (0, 3115)	1
  (0, 2616)	1
  (0, 4886)	1
  (0, 12386)	2
  (0, 14235)	1
  (0, 2755)	1
  (0, 12299)	1
  (0, 11517)	1
  (0, 14561)	1
  (0, 14820)	1
  (0, 11490)	1
  (0, 12134)	1
  (0, 14291)	1
  (0, 12567)	1
  (0, 7496)	1
  (0, 8831)	1
  (0, 11217)	1
  (0, 86)	1
  (0, 144)	1
  (0, 4435)	1
  (0, 11745)	1
  (0, 4566)	1
  (0, 6542)	1
  (0, 2061)	1
  (1, 86)	1
  :	:
  (4801, 10069)	1
  (4801, 5844)	1
  (4801, 252)	1
  (4801, 4098)	1
  (4801, 14796)	1
  (4801, 11361)	1
  (4801, 2978)	1
  (4801, 12036)	1
  (4801, 6138)	1
  (4802, 9659)	1
  (4802, 3812)	1
  (4802, 1788)	2
  (4802, 4210)	1
  (4802, 5181)	1
  (4802, 2912)	1
  (4802, 3821)	1
  (4802, 1069)	1
  (4802, 11185)	1
  (4802, 3681)	1
  (4802, 5399)	1
  (4802, 3894)	1
  (4802, 2056)	1
  (4802, 3093)	1
  (4802, 4502)	1
  (4802, 5900)	2


* CSR(Compressed Sparse Row) Matrix : sparse 한 matrix에서 0이 아닌 유효 데이터로 채워짐 + 좌표 정보로만 구성 메모리 사용량 최소화 + sparse matrix와 동일한 행렬을 표현하게 하는 데이터 구조
* consine_similarity matrix 구하기

In [13]:
cosine_sim = cosine_similarity(count_matrix)
print(cosine_sim)
print(cosine_sim.shape)

[[1.         0.10540926 0.12038585 ... 0.         0.         0.        ]
 [0.10540926 1.         0.0761387  ... 0.03651484 0.         0.        ]
 [0.12038585 0.0761387  1.         ... 0.         0.11145564 0.        ]
 ...
 [0.         0.03651484 0.         ... 1.         0.         0.04264014]
 [0.         0.         0.11145564 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.04264014 0.         1.        ]]
(4803, 4803)


### 추천

In [14]:
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]

movie_user_likes = "Avatar"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index]))

sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

i=0
print(movie_user_likes+"와 비슷한 영화 3편은 "+"\n")
for item in sorted_similar_movies:
    print(get_title_from_index(item[0]))
    i=i+1
    if i==3:
        break

Avatar와 비슷한 영화 3편은 

Guardians of the Galaxy
Aliens
Star Wars: Clone Wars: Volume 1


## 협업 필터링

### 설명

* 협업 필터링(Collaborative Filtering) : 과거의 사용자의 행동 양식(User Behavior) 데이터를 기반으로 추천하는 방식.
* 사용자 기반, 아이템 기반 : 유사도를 계산하는 방식
  - 평점행렬로 변환한 후, 유사도를 계산하여 추천.
* 잠재요인(latent factor) : 행렬의 인수분해(matrix factorization)을 이용해 잠재요인을 분석.
  - 이 평점행렬을 분해하여 더 많은 정보를 고려하는 것.

### 사용자 기반

* 당신과 비슷한 고객들이 다음 상품들을 구매.

### 아이템 기반

* 이 상품을 선택한 다른 고객들은 다음 상품을 구매했습니다.

## 행렬 인수분해

### 설명

* 잠재요인 협업 필터링은 평점행렬을 행렬 인수분해를 통해 잠재요인을 분석.
* 행렬 인수분해
1. SVD(Singular Vector Decomposition)
2. ALS(Alternating Least Squares)
3. NMF(Non-negative Matrix Factorization)

### SVD

* SVD(Singular Vector Decomposition) : 특잇값 분해
* M X N 인 행렬 A를 다음과 같이 분해
* 행렬의 정보량을 여러 layer로 쪼개서 생각할 수 있게 해줌
* 모든 m*n 행렬에 사용 가능
* 이렇게 나눌 수 있는 것을 알고 어디에 적용할 수 있나 알아본 것

$$A = U\sum V^{T}$$

* A :  rectangular matrix
* U :  orthogonal matrix
* $\sum$ : diagonal matrix
* V :  orthogonal matrix

* orthogonal matrix(직교행렬)
* $UU^{T} = U^{T}U = I$
* $U^{-1} = U^{T}$

* diagonal matrix(대각행렬)
* 대각성분을 제외한 나머지 원소값이 모두 0

In [15]:
import numpy as np
from numpy.linalg import svd

In [16]:
np.random.seed(30)
A = np.random.randint(0, 100, size=(4, 4))
A

array([[37, 37, 45, 45],
       [12, 23,  2, 53],
       [17, 46,  3, 41],
       [ 7, 65, 49, 45]])

In [17]:
svd(A)

(array([[-0.54937068, -0.2803037 , -0.76767503, -0.1740596 ],
        [-0.3581157 ,  0.69569442, -0.13554741,  0.60777407],
        [-0.41727183,  0.47142296,  0.28991733, -0.72082768],
        [-0.6291496 , -0.46389601,  0.55520257,  0.28411509]]),
 array([142.88131188,  39.87683209,  28.97701433,  14.97002405]),
 array([[-0.25280963, -0.62046326, -0.4025583 , -0.6237463 ],
        [ 0.06881225, -0.07117038, -0.8159854 ,  0.56953268],
        [-0.73215039,  0.61782756, -0.23266002, -0.16767299],
        [-0.62873522, -0.47775436,  0.34348792,  0.50838848]]))

In [18]:
U, Sigma, VT = svd(A)

print('U matrix: {}\n'.format(U.shape),U)
print('Sigma: {}\n'.format(Sigma.shape),Sigma)
print('V Transpose matrix: {}\n'.format(VT.shape),VT)

U matrix: (4, 4)
 [[-0.54937068 -0.2803037  -0.76767503 -0.1740596 ]
 [-0.3581157   0.69569442 -0.13554741  0.60777407]
 [-0.41727183  0.47142296  0.28991733 -0.72082768]
 [-0.6291496  -0.46389601  0.55520257  0.28411509]]
Sigma: (4,)
 [142.88131188  39.87683209  28.97701433  14.97002405]
V Transpose matrix: (4, 4)
 [[-0.25280963 -0.62046326 -0.4025583  -0.6237463 ]
 [ 0.06881225 -0.07117038 -0.8159854   0.56953268]
 [-0.73215039  0.61782756 -0.23266002 -0.16767299]
 [-0.62873522 -0.47775436  0.34348792  0.50838848]]


In [19]:
Sigma_mat = np.diag(Sigma)

A_ = np.dot(np.dot(U, Sigma_mat), VT)
A_

array([[37., 37., 45., 45.],
       [12., 23.,  2., 53.],
       [17., 46.,  3., 41.],
       [ 7., 65., 49., 45.]])

## 행렬 인수분해와 잠재요인 필터링

* SVD를 평가행렬에 적용하여 잠재요인을 분석
* 사람들이 평점을 매기는 요인을 잠재요인으로 취급, SVD 기법을 이용해 분해 후 다시 합침